In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mqt.predictor import ml

df = pd.read_csv(ml.helper.get_path_results(), sep=",")

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 30

apply_normalization = False

# 2x2 Matrix With Mean Results and Optimization Criterion Comparison

In [ ]:
tmp_df = df[df["MQTPredictor_expected_fidelity_expected_fidelity"] >= 0]
tmp_df = tmp_df[tmp_df["MQTPredictor_critical_depth_expected_fidelity"] >= 0]
MQT_expected_fidelity = [
    tmp_df["MQTPredictor_expected_fidelity_expected_fidelity"].mean(),
    tmp_df["MQTPredictor_expected_fidelity_critical_depth"].mean(),
]
MQT_critical_depth = [
    tmp_df["MQTPredictor_critical_depth_expected_fidelity"].mean(),
    tmp_df["MQTPredictor_critical_depth_critical_depth"].mean(),
]

In [ ]:
print(np.round(MQT_expected_fidelity, 2))
print(np.round(MQT_critical_depth, 2))

## Calculate Data Points

In [ ]:
def calculate_median(row: pd.core.series.Series, column_name: str) -> float:
    columns = [col for col in row.index if col.endswith(column_name) and "MQT" not in col]
    values = [value for col, value in row[columns].items() if value != -1]
    return np.median(values) if values else None


df["expected_expected_fidelity_median"] = df.apply(calculate_median, column_name="_expected_fidelity", axis=1)
df["critical_depth_median"] = df.apply(calculate_median, column_name="_critical_depth", axis=1)
df["expected_expected_fidelity_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["expected_expected_fidelity_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]].max(), axis=1
)
df["critical_depth_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["critical_depth_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]].max(), axis=1
)

## Optionally: Normalize

In [ ]:
if apply_normalization:
    divider_column_name = "expected_expected_fidelity_max_other"

    # Get the list of column names ending with "_expected_fidelity"
    columns_to_divide = [
        col
        for col in df.columns
        if col.endswith("_expected_fidelity")
        or col
        in [
            "expected_expected_fidelity_min_other",
            "expected_expected_fidelity_max_other",
            "expected_expected_fidelity_median",
        ]
    ]

    # Iterate through each column and perform the division
    for col_name in columns_to_divide:
        df[col_name] = df[col_name].divide(df[divider_column_name])

    divider_column_name = "critical_depth_max_other"

    # Get the list of column names ending with "_expected_fidelity"
    columns_to_divide = [
        col
        for col in df.columns
        if col.endswith("_critical_depth")
        or col in ["critical_depth_min_other", "critical_depth_max_other", "critical_depth_median"]
    ]

    # Iterate through each column and perform the division
    for col_name in columns_to_divide:
        df[col_name] = df[col_name].divide(df[divider_column_name])

# Expected Fidelity

In [ ]:
predictor = ml.Predictor()

training_data = predictor.get_prepared_training_data(figure_of_merit="expected_fidelity", save_non_zero_indices=True)
indices_test = training_data.indices_test
names_list = training_data.names_list

test_benchmarks_expected_fidelity = [names_list[index_test] for index_test in indices_test]
df_filtered_expected_fidelity = df[df["file_path"].isin(test_benchmarks_expected_fidelity)]
df_filtered_expected_fidelity = df_filtered_expected_fidelity[
    df_filtered_expected_fidelity["MQTPredictor_expected_fidelity_expected_fidelity"] >= 0
]

In [ ]:
print(
    "Fid Improvement: ",
    df_filtered_expected_fidelity.loc[
        df["MQTPredictor_expected_fidelity_expected_fidelity"] != np.inf,
        "MQTPredictor_expected_fidelity_expected_fidelity",
    ].max(),
)

In [ ]:
len(df_filtered_expected_fidelity)

In [ ]:
df_filtered_expected_fidelity["MQTPredictor_expected_fidelity_expected_fidelity"].describe()

In [ ]:
plot_kind = "line"
df_filtered_and_sorted_expected_fidelity = df_filtered_expected_fidelity.sort_values(
    by=["MQTPredictor_expected_fidelity_expected_fidelity"]
)
ax = df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path",
    y="expected_expected_fidelity_max_other",
    label="Best",
    rot=90,
    kind=plot_kind,
    color="green",
    figsize=(30, 10),
)
df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path",
    y="MQTPredictor_expected_fidelity_expected_fidelity",
    label="MQT Predictor",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color="blue",
)
df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path", y="expected_expected_fidelity_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median"
)
df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path", y="expected_expected_fidelity_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red"
)

if plot_kind == "line":
    plt.xticks(
        range(len(df_filtered_and_sorted_expected_fidelity.file_path)),
        df_filtered_and_sorted_expected_fidelity.file_path,
    )

plt.xticks(
    list(range(0, len(df_filtered_and_sorted_expected_fidelity.file_path), 1)),
    ["" for i in range(0, len(df_filtered_and_sorted_expected_fidelity.file_path), 1)],
    fontsize=16,
)

plt.xlabel("Benchmarks")
plt.ylabel("Expected expected_fidelity")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0, 2, 3]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])

plt.savefig("results/expected_fidelity_plot.pdf", bbox_inches="tight")

## Top 3 expected_fidelity

In [ ]:
df_filtered_and_sorted_expected_fidelity["Rank_MQT_expected_fidelity"] = df_filtered_and_sorted_expected_fidelity.apply(
    lambda row: sum(
        1
        for col in df_filtered_and_sorted_expected_fidelity.columns
        if col.endswith("_expected_fidelity")
        and "MQT" not in col
        and row["MQTPredictor_expected_fidelity_expected_fidelity"] >= row[col]
    ),
    axis=1,
)
df_filtered_and_sorted_expected_fidelity["Rank_MQT_expected_fidelity"] = (
    len([
        col
        for col in df_filtered_and_sorted_expected_fidelity.columns
        if col.endswith("_expected_fidelity") and "MQT" not in col
    ])
    - df_filtered_and_sorted_expected_fidelity.Rank_MQT_expected_fidelity
)
plt.hist(
    df_filtered_and_sorted_expected_fidelity.Rank_MQT_expected_fidelity.values,
    bins=range(0, 15, 1),
    align="left",
    weights=np.ones(len(df_filtered_and_sorted_expected_fidelity)) / len(df_filtered_and_sorted_expected_fidelity),
)
plt.xticks(range(0, 14, 1))

In [ ]:
len(
    df_filtered_and_sorted_expected_fidelity[
        df_filtered_and_sorted_expected_fidelity["Rank_MQT_expected_fidelity"] == 0
    ]
)

In [ ]:
len(df_filtered_and_sorted_expected_fidelity)

In [ ]:
count_top3 = 0
for _index, row in df_filtered_and_sorted_expected_fidelity.iterrows():
    if row["Rank_MQT_expected_fidelity"] in [0, 1, 2]:
        count_top3 += 1

print("Percentage of Top-3:", count_top3 / len(df_filtered_and_sorted_expected_fidelity))

# Critical Depth

In [ ]:
predictor = ml.Predictor()
training_data = predictor.get_prepared_training_data(figure_of_merit="critical_depth", save_non_zero_indices=True)
indices_test = training_data.indices_test
names_list = training_data.names_list

test_benchmarks_critical_depth = [names_list[index_test] for index_test in indices_test]
df_filtered_critical_depth = df[df["file_path"].isin(test_benchmarks_critical_depth)]
df_filtered_critical_depth = df_filtered_critical_depth[
    df_filtered_critical_depth["MQTPredictor_critical_depth_critical_depth"] >= 0
]

In [ ]:
print(
    "Crit. Dep. Improvement: ",
    df_filtered_critical_depth.loc[
        df["MQTPredictor_critical_depth_critical_depth"] != np.inf, "MQTPredictor_critical_depth_critical_depth"
    ].max(),
)

In [ ]:
len(df_filtered_critical_depth)

In [ ]:
plot_kind = "line"
df_filtered_and_sorted_critical_depth = df_filtered_critical_depth.sort_values(
    by=["MQTPredictor_critical_depth_critical_depth"]
)
ax = df_filtered_and_sorted_critical_depth.plot(
    x="file_path", y="critical_depth_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(30, 10)
)
df_filtered_and_sorted_critical_depth.plot(
    x="file_path",
    y="MQTPredictor_critical_depth_critical_depth",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color="blue",
    label="MQT Predictor",
)
df_filtered_and_sorted_critical_depth.plot(
    x="file_path", y="critical_depth_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median"
)
df_filtered_and_sorted_critical_depth.plot(
    x="file_path", y="critical_depth_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red"
)


if plot_kind == "line":
    plt.xticks(
        range(len(df_filtered_and_sorted_critical_depth.file_path)), df_filtered_and_sorted_critical_depth.file_path
    )


plt.xticks(
    list(range(0, len(df_filtered_and_sorted_critical_depth.file_path), 1)),
    ["" for i in range(0, len(df_filtered_and_sorted_critical_depth), 1)],
    fontsize=18,
)

plt.xlabel("Benchmarks")
plt.ylabel("Critical Depth")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0, 2, 3]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])

plt.savefig("results/critical_depth_plot.pdf", bbox_inches="tight")

## Top 3 Critical Depth

In [ ]:
df_filtered_and_sorted_critical_depth["Rank_MQT_critical_depth"] = df_filtered_and_sorted_critical_depth.apply(
    lambda row: sum(
        1
        for col in df_filtered_and_sorted_critical_depth.columns
        if col.endswith("_critical_depth")
        and "MQT" not in col
        and row["MQTPredictor_critical_depth_critical_depth"] >= row[col]
    ),
    axis=1,
)
df_filtered_and_sorted_critical_depth["Rank_MQT_critical_depth"] = (
    len([
        col
        for col in df_filtered_and_sorted_critical_depth.columns
        if col.endswith("_critical_depth") and "MQT" not in col
    ])
    - df_filtered_and_sorted_critical_depth.Rank_MQT_critical_depth
)
plt.hist(
    df_filtered_and_sorted_critical_depth.Rank_MQT_critical_depth.values,
    bins=range(0, 15, 1),
    align="left",
    weights=np.ones(len(df_filtered_and_sorted_critical_depth)) / len(df_filtered_and_sorted_critical_depth),
)
plt.xticks(range(0, 14, 1))

In [ ]:
len(df_filtered_and_sorted_critical_depth[df_filtered_and_sorted_critical_depth["Rank_MQT_critical_depth"] == 0])

In [ ]:
len(df_filtered_and_sorted_critical_depth)

In [ ]:
count_top3 = 0
for _index, row in df_filtered_and_sorted_critical_depth.iterrows():
    if row["Rank_MQT_critical_depth"] in [0, 1, 2]:
        count_top3 += 1

# Print the total count
print("Percentage of Top-3:", count_top3 / len(df_filtered_and_sorted_critical_depth))

# GHZ Evaluation

In [ ]:
df = pd.read_csv(ml.helper.get_path_results(ghz_results=True), sep=",")
df = df[df.num_qubits < 32]
plt.rcParams["font.size"] = 50

In [ ]:
df["expected_fidelity_median"] = df.apply(calculate_median, column_name="_expected_fidelity", axis=1)
df["critical_depth_median"] = df.apply(calculate_median, column_name="_critical_depth", axis=1)
df["expected_fidelity_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["expected_fidelity_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]].max(), axis=1
)
df["critical_depth_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["critical_depth_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]].max(), axis=1
)

In [ ]:
plot_kind = "line"
df = df.sort_values(by=["num_qubits"])
ax = df.plot(
    x="file_path",
    y="expected_fidelity_max_other",
    label="Best",
    rot=90,
    kind=plot_kind,
    color="green",
    figsize=(20, 10),
)
df.plot(
    x="file_path",
    y="MQTPredictor_expected_fidelity_expected_fidelity",
    label="MQT Predictor",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color="blue",
)
df.plot(x="file_path", y="expected_fidelity_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median")
df.plot(x="file_path", y="expected_fidelity_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Number of Qubits")
plt.ylabel("Expected Fidelity")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0, 2, 3]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])


plt.savefig("results/expected_fidelity_ghz_plot.pdf", bbox_inches="tight")

In [ ]:
plot_kind = "line"
df = df.sort_values(by=["num_qubits"])
ax = df.plot(
    x="file_path", y="critical_depth_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(20, 10)
)
df.plot(
    x="file_path",
    y="MQTPredictor_critical_depth_critical_depth",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color="blue",
    label="MQT Predictor",
)
df.plot(x="file_path", y="critical_depth_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Number of Qubits")
plt.ylabel("Critical Depth")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0, 2]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])

plt.savefig("results/critical_depth_ghz_plot.pdf", bbox_inches="tight")